In [5]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
!pip install transformers requests beautifulsoup4 pandas numpy
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [6]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/638M [00:00<?, ?B/s]

In [19]:
import numpy as np
import pandas as pd

df_train = pd.read_csv('Corona_NLP_train.csv' , encoding = "ISO-8859-1", index_col = False)
df_train = df_train.drop(columns = ['UserName' , 'ScreenName' , 'Location' , 'TweetAt'])
sentiment_list = ['Extremely Negative','Negative','Neutral','Positive','Extremely Positive'] 
df_train['Sentiment'] = df_train['Sentiment'].apply(lambda x: sentiment_list.index(x)+1)
df_train = df_train[:500]
df_train.head(5)

,OriginalTweet,Sentiment
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,3
1,advice Talk to your neighbours family to excha...,4
2,Coronavirus Australia: Woolworths to give elde...,4
3,My food stock is not the only one which is emp...,4
4,"Me, ready to go at supermarket during the #COV...",1


In [20]:

def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    #print(int(torch.argmax(result.logits))+1)
    return int(torch.argmax(result.logits))+1

df_train['Sentiment_pred'] = df_train['OriginalTweet'].apply(lambda x: sentiment_score(x))
df_train.head(50)

,OriginalTweet,Sentiment,Sentiment_pred
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,3,1
1,advice Talk to your neighbours family to excha...,4,4
2,Coronavirus Australia: Woolworths to give elde...,4,4
3,My food stock is not the only one which is emp...,4,5
4,"Me, ready to go at supermarket during the #COV...",1,1
5,As news of the regionÂs first confirmed COVID...,4,1
6,Cashier at grocery store was sharing his insig...,4,4
7,Was at the supermarket today. Didn't buy toile...,3,1
8,Due to COVID-19 our retail store and classroom...,4,1
9,"For corona prevention,we should stop to buy th...",2,3


In [21]:
comparison_column = np.where(df_train['Sentiment_pred'] == df_train['Sentiment'], True, False)
df_train["results"] = comparison_column
detected_tru = df_train["results"].value_counts()[True]
detected_false = df_train["results"].value_counts()[False]
print(detected_tru, detected_false)

118 382
